LSTM + N - Gram model


In [ ]:

import re
import numpy as np
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.layers import LSTM, Dropout
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import RMSprop
import matplotlib
matplotlib.use('agg')
import heapq
def compute_n_gram(words, n):
    return [words[i:i+n] for i in range(len(words)-n+1)]
def compute_n_gram_with_next_word(words, n):
    n_gram = compute_n_gram(words, n)[:-1]
    next_words = [words[i+n] for i in range(len(n_gram))]
    return n_gram, next_words
path = '/content/Final.txt'
text = open(path, encoding='utf-8', errors='ignore').read().lower()
text = re.sub(r'[^\w\s]', '', text) 
words = text.split()
vocab = sorted(list(set(words))) 
vocab_size = len(vocab)
word_indices = dict((c, i) for i, c in enumerate(vocab))
indices_words = dict((i, c) for i, c in enumerate(vocab))
SEQUENCE_LENGTH = 9
bigrams, next_word_bigrams = compute_n_gram_with_next_word(words, 2)
trigrams, next_word_trigrams = compute_n_gram_with_next_word(words, 3)
sentences = bigrams 
print(sentences)
next_words = next_word_bigrams + next_word_trigrams
X = np.zeros((len(sentences), SEQUENCE_LENGTH, len(vocab)), dtype=np.bool)
y = np.zeros((len(sentences), len(vocab)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, word in enumerate(sentence):
        X[i, t, word_indices[word]] = 1
    y[i, word_indices[next_words[i]]] = 1
model = Sequential()
model.add(LSTM(128, input_shape=(SEQUENCE_LENGTH, len(vocab))))
model.add(Dense(len(vocab)))
model.add(Activation('softmax'))
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
history = model.fit(X, y, validation_split=0.05, batch_size=128, epochs=500, shuffle=True).history
def prepare_input(text):
    x = np.zeros((1, SEQUENCE_LENGTH, len(vocab)))
    words = text.split()
    for t, word in enumerate(words):
        if word in word_indices:
            x[0, t, word_indices[word]] = 1
    return x
def sample(preds, top_n):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds)
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    return heapq.nlargest(top_n, range(len(preds)), preds.take)
def predict_completion(text):
    prediction = ''
    prev=""
    while True:
        x = prepare_input(text)
        preds = model.predict(x, verbose=0)[0]
        next_index = sample(preds, top_n=1)[0]
        next_word = indices_words[next_index]

        if prev==next_word or next_word == ' ' or len(prediction.split()) > SEQUENCE_LENGTH:
            return prediction
        prev=next_word
        prediction += next_word + ' '
        text = ' '.join(prediction.split()[-SEQUENCE_LENGTH:])


[['prologue', 'the'], ['the', 'day'], ['day', 'was'], ['was', 'grey'], ['grey', 'and'], ['and', 'bitter'], ['bitter', 'cold'], ['cold', 'and'], ['and', 'the'], ['the', 'dogs'], ['dogs', 'would'], ['would', 'not'], ['not', 'take'], ['take', 'the'], ['the', 'scent'], ['scent', 'the'], ['the', 'big'], ['big', 'black'], ['black', 'bitch'], ['bitch', 'had'], ['had', 'taken'], ['taken', 'one'], ['one', 'sniff'], ['sniff', 'at'], ['at', 'the'], ['the', 'bear'], ['bear', 'tracks'], ['tracks', 'backed'], ['backed', 'off'], ['off', 'and'], ['and', 'skulked'], ['skulked', 'back'], ['back', 'to'], ['to', 'the'], ['the', 'pack'], ['pack', 'with'], ['with', 'her'], ['her', 'tail'], ['tail', 'between'], ['between', 'her'], ['her', 'legs'], ['legs', 'the'], ['the', 'dogs'], ['dogs', 'huddled'], ['huddled', 'together'], ['together', 'miserably'], ['miserably', 'on'], ['on', 'the'], ['the', 'riverbank'], ['riverbank', 'as'], ['as', 'the'], ['the', 'wind'], ['wind', 'snapped'], ['snapped', 'at'], ['at', 

<ipython-input-21-cecdbbf47cb7>:32: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  X = np.zeros((len(sentences), SEQUENCE_LENGTH, len(vocab)), dtype=np.bool)
<ipython-input-21-cecdbbf47cb7>:33: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.zeros((len(sentences), len(vocab)), dtype=np.bool)


Epoch 1/500
80/80 [==============================] - 3s 15ms/step - loss: 7.0497 - accuracy: 0.0576 - val_loss: 6.6810 - val_accuracy: 0.0428
Epoch 2/500
80/80 [==============================] - 1s 10ms/step - loss: 6.4318 - accuracy: 0.0587 - val_loss: 6.6645 - val_accuracy: 0.0428
Epoch 3/500
80/80 [==============================] - 1s 11ms/step - loss: 6.3836 - accuracy: 0.0587 - val_loss: 6.6404 - val_accuracy: 0.0428
Epoch 4/500
80/80 [==============================] - 1s 9ms/step - loss: 6.3554 - accuracy: 0.0587 - val_loss: 6.6616 - val_accuracy: 0.0428
Epoch 5/500
80/80 [==============================] - 1s 7ms/step - loss: 6.3314 - accuracy: 0.0587 - val_loss: 6.6381 - val_accuracy: 0.0428
Epoch 6/500
80/80 [==============================] - 1s 8ms/step - loss: 6.3128 - accuracy: 0.0587 - val_loss: 6.6386 - val_accuracy: 0.0428
Epoch 7/500
80/80 [==============================] - 1s 9ms/step - loss: 6.2994 - accuracy: 0.0587 - val_loss: 6.7580 - val_accuracy: 0.0428
Epoch 8/50

Output for LSTM + N-gram model


In [ ]:
def predict_completions(text, n):
    x = prepare_input(text)
    preds = model.predict(x, verbose=0)[0]
    next_indices = sample(preds, n)
    return [indices_words[idx] +" "+predict_completion(text + ' ' + indices_words[idx]) for idx in next_indices]
print(predict_completions("dolorous",4))

['words to that one do a come ', 'commander jon who jaime ', 'frey swords catelyns if ', 'leather edd look i said no look ']


Implementation of The jaccardian Similarity


In [ ]:
from transformers import pipeline
with open("/content/GOT.txt",encoding='utf-8',errors='ignore') as myfile:
    context = myfile.read().lower()
question_answerer = pipeline("question-answering", model='distilbert-base-uncased-distilled-squad')
def new():
  global final
  final=input("What is your question:")
  return final
result = question_answerer(question=new(),context=context)
res=result['answer']
print(f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('words')
import re
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
with open('/content/GOT.txt', 'r', errors="ignore") as f:
    text_data = f.read()
sentences = re.split('[.?!]', text_data.lower())
sentences = [sentence.strip().lower() for sentence in sentences if sentence.strip()]
input_text = "smallwood"
input_words = input_text.lower().split()
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(sentences)
vocab = list(vectorizer.vocabulary_.keys())
input_set = set(input_words)
similarity = []
for sentence in sentences:
    sentence_set = set(sentence.lower().split())
    jaccard_sim = len(input_set.intersection(sentence_set)) / len(input_set.union(sentence_set))
    similarity.append(jaccard_sim)
defining_indices = np.argsort(similarity)[::-1][:10]
defining_sentences=[]
for i in defining_indices:
  defining_sentences.append(sentences[i])
  if i-1>=0:
    defining_sentences.append(sentences[i-1])
  if i+1<len(sentences):
    defining_sentences.append(sentences[i+1])
new_result=[]
for i, sentence in enumerate(sentences):
    if i in defining_indices:
        continue
    elif input_text.lower() in sentence.lower() and i >=0:
        new_result.append(sentences[i])
        new_result.append(sentences[i-1])
        if i+1<len(sentences):
          new_result.append(sentences[i+1])
    elif input_text.lower() in sentence.lower():
        new_result.append(sentences[i])
        if i+1<len(sentences):
          new_result.append(sentences[i+1])
new_result=defining_sentences
if new_result==[]:
  defining_indices = np.argsort(similarity)[::-1]
  defining_sentences = [sentences[i] for i in defining_indices]
  new_result=[]
  for i, sentence in enumerate(sentences):
    if input_text.lower() in sentence.lower() and i >=0:
      new_result.append(sentences[i])
      new_result.append(sentences[i-1])
      if i+1<len(sentences):
        new_result.append(sentences[i+1])
    elif input_text.lower() in sentence.lower():
      new_result.append(sentences[i])
      if i+1<len(sentences):
        new_result.append(sentences[i+1])
import spacy
nlp = spacy.load("en_core_web_sm")
def join_sentences(sentences):
    text = " ".join(sentences)
    doc = nlp(text)
    sents = list(doc.sents)
    result = ""
    for i, sent in enumerate(sents):
        if i < len(sents) - 1:
            result += sent.text.strip() + ","
        else:
            result += sent.text.strip()
        if i == len(sents) - 2:
            result += " and "
    return result
joined = join_sentences(new_result)
print(joined)


What is your question:Who sent Bennen?
